In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# تولید داده‌های مصنوعی
np.random.seed(42)
num_samples = 20000  # افزایش تعداد داده‌ها برای بهبود مدل

customer_id = np.random.randint(1000, 9999, num_samples)
call_duration = np.random.randint(30, 900, num_samples)
call_reason = np.random.choice(["مشکل اینترنت", "پشتیبانی", "مالی", "شکایت"], num_samples)
agent_id = np.random.randint(1, 50, num_samples)
response_time = np.random.randint(5, 180, num_samples)

def generate_satisfaction(call_duration, response_time, call_reason):
    score = 3
    if call_duration < 60:
        score = np.random.choice([1, 2], p=[0.6, 0.4])
    elif response_time > 120:
        score = np.random.choice([1, 2, 3], p=[0.5, 0.3, 0.2])
    elif call_reason == "شکایت":
        score = np.random.choice([1, 2, 3], p=[0.5, 0.3, 0.2])
    elif call_reason in ["پشتیبانی", "مالی"]:
        score = np.random.choice([3, 4, 5], p=[0.2, 0.3, 0.5])
    return score

satisfaction_score = [generate_satisfaction(call_duration[i], response_time[i], call_reason[i]) for i in range(num_samples)]

df = pd.DataFrame({
    "customer_id": customer_id,
    "call_duration": call_duration,
    "call_reason": call_reason,
    "agent_id": agent_id,
    "response_time": response_time,
    "satisfaction_score": satisfaction_score
})

# اضافه کردن ویژگی‌های جدید
df["day_of_week"] = np.random.randint(0, 7, len(df))
df["call_hour"] = np.random.randint(8, 22, len(df))
agent_avg_response_time = {agent: np.random.randint(5, 150) for agent in df["agent_id"].unique()}
df["agent_avg_response_time"] = df["agent_id"].map(agent_avg_response_time)
df["response_to_duration_ratio"] = df["response_time"] / df["call_duration"]
df["agent_avg_satisfaction"] = df.groupby("agent_id")["satisfaction_score"].transform("mean")

# تبدیل داده‌های دسته‌ای به عددی
df["call_reason"] = LabelEncoder().fit_transform(df["call_reason"])

# نرمال‌سازی داده‌ها
scaler = StandardScaler()
numeric_features = ["call_duration", "response_time", "agent_avg_response_time", "response_to_duration_ratio", "agent_avg_satisfaction"]
df[numeric_features] = scaler.fit_transform(df[numeric_features])

# تنظیم خروجی مدل
y = df["satisfaction_score"] - 1  # تبدیل بازه 1-5 به 0-4
X = df.drop(columns=["satisfaction_score", "customer_id"])

# تقسیم داده‌ها
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# تنظیمات بهینه RandomForest
param_grid_rf = {
    'n_estimators': [100, 300],
    'max_depth': [8, 12],
    'min_samples_split': [2, 5]
}
grid_search_rf = GridSearchCV(RandomForestClassifier(random_state=42, class_weight='balanced'), param_grid_rf, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)
best_rf_params = grid_search_rf.best_params_

# تنظیمات بهینه XGBoost
param_grid_xgb = {
    'n_estimators': [100, 300, 500],
    'max_depth': [6, 10, 14],
    'learning_rate': [0.05, 0.1, 0.2],
    'scale_pos_weight': [1, 2]
}
grid_search_xgb = GridSearchCV(XGBClassifier(random_state=42), param_grid_xgb, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_xgb.fit(X_train, y_train)
best_xgb_params = grid_search_xgb.best_params_

# مدل LightGBM
lgb_model = LGBMClassifier(n_estimators=300, max_depth=12, learning_rate=0.1, random_state=42)

# مدل‌ها با تنظیمات بهینه
rf_model = RandomForestClassifier(**best_rf_params, class_weight='balanced', random_state=42)
xgb_model = XGBClassifier(**best_xgb_params, random_state=42)

# اجرای Ensemble Learning
ensemble_model = VotingClassifier(estimators=[
    ('rf', rf_model),
    ('xgb', xgb_model),
    ('lgb', lgb_model)
], voting='soft', weights=[1, 1, 3])

ensemble_model.fit(X_train, y_train)
y_pred_ensemble = ensemble_model.predict(X_test)

# نمایش دقت نهایی
accuracy = accuracy_score(y_test, y_pred_ensemble)
print(f"🔹 Accuracy با Ensemble Learning: {accuracy:.2f}")


/home/behnam/anaconda3/envs/irancell/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:07:28] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/behnam/anaconda3/envs/irancell/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:07:28] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/behnam/anaconda3/envs/irancell/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:07:28] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/behnam/anaconda3/envs/irancell/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:07:28] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/behnam/anaconda3/envs/irancell/lib/python3.9/site-

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000203 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 853
[LightGBM] [Info] Number of data points in the train set: 16000, number of used features: 9
[LightGBM] [Info] Start training from score -1.336790
[LightGBM] [Info] Start training from score -1.815924
[LightGBM] [Info] Start training from score -1.152211
[LightGBM] [Info] Start training from score -2.322150
[LightGBM] [Info] Start training from score -1.828683
🔹 Accuracy با Ensemble Learning: 0.54
